In [113]:
# Load required libraries
library(tidyverse)
library(Hmisc)
library(VIM)
library(mice)

#library(ggplot2)
#library(dplyr)
#library(tidyr)
library(knitr)

### Load Data

In [204]:
# Baca data
data <- read.csv("DATA_MASTER - MASTER_RAPI.csv")

In [205]:
# Lihat Kolom Dalam Data
colnames(data)

[1] "NO"               "KODE_SAMPEL"      "NO_RM"            "NAMA"            
 [5] "TGL_LAHIR"        "USIA"             "JENIS_KEL"        "KEL_UTAMA"       
 [9] "TGL_OPNAME"       "TGL_CATH"         "RIW_DM"           "RIW_HT"          
[13] "RIW_DISLIPIDEMIA" "RIW_MEROKOK"      "RIW_KELUARGA"     "RIW_ALKOHOL"     
[17] "TB"               "BB"               "TD_S"             "TD_D"            
[21] "HR"               "RR"               "TEMP"             "KEPALA_LEHER"    
[25] "JANTUNG"          "PARU"             "ABDOMEN"          "EKSTREMITAS"     
[29] "HB"               "LEUKOSIT"         "PLT"              "HBSAG"           
[33] "BUN"              "SK"               "ALBUMIN"          "ANTIHCV"         
[37] "NATRIUM"          "K"                "GDS"              "GDP"             
[41] "GD_2JPP"          "KOL_TOT"          "TG"               "HDL"             
[45] "LDL"              "UREUM"            "EGFR"             "ASAM_URAT"       
[49] "HBA1C"            "HSTROP_1"         "IRAMA_EKG"        "AKSIS_EKG"       
[53] "X"                "EF_ECHO"          "X.1"              "FOTO_THORAX"     
[57] "X.2"              "LMCA"             "LMCA_POINT"       "LAD_PROX"        
[61] "LAD_PROX_POINT"   "LAD_MID"          "LAD_MID_POINT"    "LAD_DIST"        
[65] "LAD_DIST_POINT"   "LAD_D1"           "LAD_D1_POINT"     "LAD_D2"          
[69] "LAD_D2_POINT"     "LAD_API"          "LAD_API_POINT"    "LCX_PROX"        
[73] "LCX_PROX_POINT"   "LCX_DIST"         "LCX_DIST_POINT"   "LCX_OM"          
[77] "LCX_OM_POINT"     "LCX_PL"           "LCX_PL_POINT"     "RCA_PROX"        
[81] "RCA_PROX_POINT"   "RCA_MID"          "RCA_MID_POINT"    "RCA_DIST"        
[85] "RCA_DIST_POINT"   "RCA_PD"           "RCA_PD_POINT"     "GENSINI"         
[89] "SEG_TERLIBAT"

In [116]:
# Lihat struktur data
str(data)

'data.frame':	245 obs. of  89 variables:
 $ NO              : int  1 2 3 4 5 6 7 8 9 10 ...
 $ KODE_SAMPEL     : int  24211 24212 24213 24214 24215 24216 24217 24218 24219 24220 ...
 $ NO_RM           : int  264773 263972 266650 266306 263608 266225 266033 266454 267372 243943 ...
 $ NAMA            : chr  "IJAH" "IRWANSYAH" "HAJI FATHUL AZIS" "IDRIS" ...
 $ TGL_LAHIR       : chr  "31/12/1940" "8/1/1978" "07/12/1959" "1/7/1967" ...
 $ USIA            : int  83 46 64 57 83 78 57 65 65 50 ...
 $ JENIS_KEL       : chr  "P" "L" "L" "L" ...
 $ KEL_UTAMA       : chr  "NYERI DADA HILANG TIMBUL, RUJUKAN RSHK" "NYERI PERUT TITAPI TIDAK ADA NYERI DADA" "NYERI DADA, SESAK NAFAS, LEMAS CEPAT LELAH" "NYERI DADA SAAT BERAKTIVITAS" ...
 $ TGL_OPNAME      : chr  "31/07/2024" "31/07/2024" "19/08/2024" "19/08/2024" ...
 $ TGL_CATH        : chr  "01/08/2024" "01/08/2024" "21/08/2024" "20/08/2024" ...
 $ RIW_DM          : chr  "TIDAK" "YA" "TIDAK" "TIDAK" ...
 $ RIW_HT          : chr  "TIDAK" "YA" "TIDAK"

In [117]:
summary(data$K)

   Length     Class      Mode 
      245 character character 

### Konversi Data

In [118]:
# Fungsi untuk mengkonversi floating point dengan koma
clean_numeric <- function(x) {
  # Jika kolom berisi karakter atau faktor
  if(is.character(x) || is.factor(x)) {
    # Ganti koma dengan titik
    x <- gsub(",", ".", x)
    # Coba konversi ke numerik
    x <- suppressWarnings(as.numeric(x))
  }
  return(x)
}

# Identifikasi kolom yang berpotensi numerik
numeric_cols <- c("USIA", "TB", "BB", "TD_S", "TD_D", "HR", "RR", "TEMP",
                 "HB", "LEUKOSIT", "PLT", "BUN", "SK", "ALBUMIN", "NATRIUM", "K",
                 "GDS", "GDP", "GD_2JPP", "KOL_TOT", "TG", "HDL", "LDL",
                 "UREUM", "EGFR", "ASAM_URAT", "HBA1C", "HSTROP_1", "LMCA_POINT", 
                 "LAD_PROX_POINT", "LAD_MID_POINT", "LAD_DIST_POINT", "LAD_D1_POINT",
                 "LAD_D2_POINT", "LAD_API_POINT", "LCX_PROX_POINT", "LCX_DIST_POINT",
                 "LCX_OM_POINT", "LCX_PL_POINT", "RCA_PROX_POINT", "RCA_MID_POINT", 
                 "RCA_DIST_POINT", "RCA_PD_POINT", "GENSINI")

# Aplikasikan konversi
data[numeric_cols] <- lapply(data[numeric_cols], clean_numeric)

In [119]:
# Konversi variabel biner menjadi numerik
data$JENIS_KEL <- ifelse(data$JENIS_KEL == "L", 1, 0)
data$RIW_DM <- ifelse(data$RIW_DM == "YA", 1, 0)
data$RIW_HT <- ifelse(data$RIW_HT == "YA", 1, 0)
data$RIW_DISLIPIDEMIA <- ifelse(data$RIW_DISLIPIDEMIA == "YA", 1, 0)
data$RIW_MEROKOK <- ifelse(data$RIW_MEROKOK == "YA", 1, 0)
data$RIW_KELUARGA <- ifelse(data$RIW_KELUARGA == "YA", 1, 0)
data$RIW_ALKOHOL <- ifelse(data$RIW_ALKOHOL == "YA", 1, 0)
data$HBSAG <- ifelse(data$HBSAG == "REAKTIF", 1, 0)
data$ANTIHCV <- ifelse(data$ANTIHCV == "REAKTIF", 1, 0)

In [120]:
# Melihat Struktur Data Setelah Konversi
str(data)

'data.frame':	245 obs. of  89 variables:
 $ NO              : int  1 2 3 4 5 6 7 8 9 10 ...
 $ KODE_SAMPEL     : int  24211 24212 24213 24214 24215 24216 24217 24218 24219 24220 ...
 $ NO_RM           : int  264773 263972 266650 266306 263608 266225 266033 266454 267372 243943 ...
 $ NAMA            : chr  "IJAH" "IRWANSYAH" "HAJI FATHUL AZIS" "IDRIS" ...
 $ TGL_LAHIR       : chr  "31/12/1940" "8/1/1978" "07/12/1959" "1/7/1967" ...
 $ USIA            : int  83 46 64 57 83 78 57 65 65 50 ...
 $ JENIS_KEL       : num  0 1 1 1 1 0 1 0 1 1 ...
 $ KEL_UTAMA       : chr  "NYERI DADA HILANG TIMBUL, RUJUKAN RSHK" "NYERI PERUT TITAPI TIDAK ADA NYERI DADA" "NYERI DADA, SESAK NAFAS, LEMAS CEPAT LELAH" "NYERI DADA SAAT BERAKTIVITAS" ...
 $ TGL_OPNAME      : chr  "31/07/2024" "31/07/2024" "19/08/2024" "19/08/2024" ...
 $ TGL_CATH        : chr  "01/08/2024" "01/08/2024" "21/08/2024" "20/08/2024" ...
 $ RIW_DM          : num  0 1 0 0 0 0 0 0 0 0 ...
 $ RIW_HT          : num  0 1 0 1 0 0 1 0 0 1 ...
 

In [121]:
# Membuktikan Bahwa Variabel Sudah Bertipe Numerik
summary(data$K)
summary(data$GENSINI)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  2.500   3.800   4.200   5.249   4.600 112.000      14 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   0.00   17.00   39.00   47.48   72.00  224.00 

In [122]:
# Parameter Ini Tidak Digunakan Saat Mengolah Data
parameter_eksklusi_olah_data <- c(
               "NO",
               "KODE_SAMPEL",
               "NO_RM",
               "NAMA",
               "TGL_LAHIR",
               "KEL_UTAMA",
               "TGL_OPNAME",
               "TGL_CATH",
               "HSTROP_1",
               "X",
               "X.1",
               "X.2",
               "AKSIS_EKG",
               "IRAMA_EKG",
               "EF_ECHO",
               "FOTO_THORAX")

data <- data[!names(data) %in% parameter_eksklusi_olah_data]

### Analisis _Missing Data_

In [206]:
# Hitung persentase missing untuk setiap kolom, diurutkan desc() dari persentase data hilang
missing_summary <- data.frame(
  Variable = names(data),
  Missing_Count = colSums(is.na(data)),
  Missing_Percent = round(colSums(is.na(data))/nrow(data)*100, 2)
) %>%
  arrange(desc(Missing_Percent))

# Tulis Hasil Missing Data Ke CSV
kable(missing_summary, row.names = FALSE)
write.csv(missing_summary, "./out/missing_summary.csv", quote = FALSE, row.names= FALSE)



|Variable         | Missing_Count| Missing_Percent|
|:----------------|-------------:|---------------:|
|KEPALA_LEHER     |           245|          100.00|
|JANTUNG          |           245|          100.00|
|PARU             |           245|          100.00|
|ABDOMEN          |           245|          100.00|
|EKSTREMITAS      |           245|          100.00|
|IRAMA_EKG        |           245|          100.00|
|AKSIS_EKG        |           245|          100.00|
|X                |           245|          100.00|
|EF_ECHO          |           245|          100.00|
|X.1              |           245|          100.00|
|FOTO_THORAX      |           245|          100.00|
|X.2              |           245|          100.00|
|GD_2JPP          |           231|           94.29|
|GDP              |           149|           60.82|
|TG               |           138|           56.33|
|LDL              |           134|           54.69|
|HDL              |           133|           54.29|
|KOL_TOT  

In [124]:
# Visualisasi pola missing data untuk variabel utama
# Buka device PNG dengan ukuran yang lebih besar
png("./out/missing_summary.png", 
    width = 3200,    # Lebar dalam pixel
    height = 1800,   # Tinggi dalam pixel
    res = 300)       # Resolusi

# Plot
aggr(data, 
     numbers = TRUE,
     bars = FALSE,
     combined = TRUE,
     border = TRUE,
     cex.axis = 0.5,     # Ukuran label sedikit diperbesar
     horizontal = TRUE,
     las = 2,
     mar = c(4, 12, 4, 2)  # Margin kiri diperbesar untuk label
)

# Tutup device
dev.off()

Warning message in plot.aggr(res, ...):
"not enough vertical space to display frequencies (too many combinations)"


pdf 
  2

### Eksklusi Variabel (Bagian 1)

In [125]:
# Drop kolom berdasarkan kriteria missing value >50% dan relevansi klinis
columns_to_drop <- c(
    # Kolom dengan missing value tinggi
    "TEMP", "ALBUMIN", "GD_2JPP", "EGFR", "ASAM_URAT", 
    "KEPALA_LEHER", "JANTUNG", "PARU", "ABDOMEN", "EKSTREMITAS",
    
    # Kolom yang merupakan hasil angiografi (tidak bisa jadi prediktor)
    "LMCA", "LMCA_POINT", 
    "LAD_PROX", "LAD_PROX_POINT", "LAD_MID", "LAD_MID_POINT",
    "LAD_DIST", "LAD_DIST_POINT", "LAD_D1", "LAD_D1_POINT",
    "LAD_D2", "LAD_D2_POINT", "LAD_API", "LAD_API_POINT",
    "LCX_PROX", "LCX_PROX_POINT", "LCX_DIST", "LCX_DIST_POINT",
    "LCX_OM", "LCX_OM_POINT", "LCX_PL", "LCX_PL_POINT",
    "RCA_PROX", "RCA_PROX_POINT", "RCA_MID", "RCA_MID_POINT",
    "RCA_DIST", "RCA_DIST_POINT", "RCA_PD", "RCA_PD_POINT",
    "SEG_TERLIBAT"

    # Nilai tinggi tetapi tidak di drop, pertimbangan klinis:
    # HBA1C, GDP, TG, LDL, HDL, KOLESTEROL TOTAL, UREUM,
)

# Membuat dataset baru tanpa kolom yang di-drop
data_clean <- data[, !names(data) %in% columns_to_drop]

# 2. Melihat struktur data setelah pembersihan
str(data_clean)

'data.frame':	245 obs. of  32 variables:
 $ USIA            : int  83 46 64 57 83 78 57 65 65 50 ...
 $ JENIS_KEL       : num  0 1 1 1 1 0 1 0 1 1 ...
 $ RIW_DM          : num  0 1 0 0 0 0 0 0 0 0 ...
 $ RIW_HT          : num  0 1 0 1 0 0 1 0 0 1 ...
 $ RIW_DISLIPIDEMIA: num  0 0 0 0 0 0 0 0 0 0 ...
 $ RIW_MEROKOK     : num  1 0 0 1 0 0 0 0 0 0 ...
 $ RIW_KELUARGA    : num  0 0 0 0 0 1 0 0 0 0 ...
 $ RIW_ALKOHOL     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ TB              : int  NA 161 NA 155 165 150 NA 150 155 168 ...
 $ BB              : int  NA 72 NA 55 60 55 NA 65 50 70 ...
 $ TD_S            : int  NA 143 104 150 NA 110 167 96 100 179 ...
 $ TD_D            : int  NA 85 66 98 NA 70 107 57 64 116 ...
 $ HR              : int  NA 93 59 70 NA 63 79 81 52 71 ...
 $ RR              : int  NA 20 20 20 NA 28 22 20 28 28 ...
 $ HB              : num  9.4 14.5 15.5 13.5 14.5 12.3 12.3 11.6 12.8 15.3 ...
 $ LEUKOSIT        : num  8.29 7.77 9.93 7.41 NA ...
 $ PLT             : num  347 231 328 170

### _Multiple Imputation_

In [126]:
# 1. Persiapan untuk multiple imputation
# Pertama lihat variabel yang akan diimputasi,
# yaitu variabel dengan data kosong (is.na) > 0
vars_for_imputation <- names(data_clean)[colSums(is.na(data_clean)) > 0]
print("Variabel yang akan diimputasi:")
print(vars_for_imputation)

[1] "Variabel yang akan diimputasi:"
 [1] "TB"       "BB"       "TD_S"     "TD_D"     "HR"       "RR"      
 [7] "HB"       "LEUKOSIT" "PLT"      "BUN"      "SK"       "NATRIUM" 
[13] "K"        "GDS"      "GDP"      "KOL_TOT"  "TG"       "HDL"     
[19] "LDL"      "UREUM"    "HBA1C"   


In [127]:
# Buat formula untuk imputasi
imputation_formula <- as.formula(paste("~", paste(vars_for_imputation, collapse = "+")))
imputation_formula

~TB + BB + TD_S + TD_D + HR + RR + HB + LEUKOSIT + PLT + BUN + 
    SK + NATRIUM + K + GDS + GDP + KOL_TOT + TG + HDL + LDL + 
    UREUM + HBA1C

In [128]:
# Lakukan multiple imputation
set.seed(123)
imputation <- aregImpute(imputation_formula, 
                        data = data_clean,
                        n.impute = 10,
                        type = 'pmm',
                        nk = 0)


Iteration 13 


In [129]:
# 2. Dapatkan dataset yang sudah diimputasi menggunakan fit.mult.impute
# Menyimpan hasil imputasi dalam list
imputed_datasets <- list()

# Loop untuk mendapatkan setiap dataset hasil imputasi
for(i in 1:10) {  # 10 sesuai dengan n.impute
    imputed_datasets[[i]] <- impute.transcan(
        imputation,
        imputation = i,
        data = data_clean,
        list.out = TRUE,
        pr = FALSE,
        check = FALSE
    )
}

In [130]:
# Menggunakan dataset pertama untuk analisis deskriptif
data_imputed <- data_clean
for(var in names(imputed_datasets[[1]])) {
    data_imputed[[var]] <- imputed_datasets[[1]][[var]]
}
# summary(data_imputed)
kable(data_imputed)



| USIA| JENIS_KEL| RIW_DM| RIW_HT| RIW_DISLIPIDEMIA| RIW_MEROKOK| RIW_KELUARGA| RIW_ALKOHOL|  TB|  BB| TD_S| TD_D|  HR| RR|   HB| LEUKOSIT|    PLT| HBSAG|   BUN|   SK| ANTIHCV| NATRIUM|      K| GDS| GDP| KOL_TOT|  TG| HDL| LDL| UREUM| HBA1C| GENSINI|
|----:|---------:|------:|------:|----------------:|-----------:|------------:|-----------:|---:|---:|----:|----:|---:|--:|----:|--------:|------:|-----:|-----:|----:|-------:|-------:|------:|---:|---:|-------:|---:|---:|---:|-----:|-----:|-------:|
|   83|         0|      0|      0|                0|           1|            0|           0| 150|  65|  153|   70|  91| 20|  9.4|    8.286| 347.40|     0| 12.15|  0.8|       0|   136.0|   3.80| 135|  86|     175|  77|  43|  45|    26|   5.6|    14.0|
|   46|         1|      1|      1|                0|           0|            0|           0| 161|  72|  143|   85|  93| 20| 14.5|    7.767| 231.00|     0| 18.22|  1.2|       0|   137.0|   4.50| 148| 126|     216| 171|  40| 186|    39|   9.1|    

### Analisis Deskriptif

#### Summary Data

In [131]:
# 3. Analisis deskriptif dengan data yang sudah diimputasi
# Definisikan variabel kategorik biner (yang sudah dikonversi ke 1/0)
binary_vars <- c("JENIS_KEL", "RIW_DM", "RIW_HT", "RIW_DISLIPIDEMIA", 
                "RIW_MEROKOK", "RIW_KELUARGA", "RIW_ALKOHOL",
                "HBSAG", "ANTIHCV")

# Definisikan variabel numerik murni
numeric_vars <- c("USIA", "BB", "TB", "TD_S", "TD_D", "HR", "RR",
                 "HB", "LEUKOSIT", "PLT", "BUN", "SK", "NATRIUM", "K",
                 "GDS", "GDP", "KOL_TOT", "TG", "HDL", "LDL",
                 "UREUM", "HBA1C", "GENSINI")

# Analisis untuk variabel numerik
numeric_summary <- data.frame(
    Variable = numeric_vars,
    N = sapply(data_imputed[numeric_vars], length),
    Mean = round(sapply(data_imputed[numeric_vars], mean), 2),
    SD = round(sapply(data_imputed[numeric_vars], sd), 2),
    Median = round(sapply(data_imputed[numeric_vars], median), 2),
    Min = round(sapply(data_imputed[numeric_vars], min), 2),
    Max = round(sapply(data_imputed[numeric_vars], max), 2),
    Q1 = round(sapply(data_imputed[numeric_vars], function(x) quantile(x, 0.25)), 2),
    Q3 = round(sapply(data_imputed[numeric_vars], function(x) quantile(x, 0.75)), 2)
)

# Analisis untuk variabel biner
binary_summary <- data.frame(
    Variable = binary_vars,
    N = sapply(data_imputed[binary_vars], length),
    Yes_n = sapply(data_imputed[binary_vars], sum),
    Yes_percent = round(sapply(data_imputed[binary_vars], function(x) mean(x) * 100), 2),
    No_n = sapply(data_imputed[binary_vars], function(x) sum(x == 0)),
    No_percent = round(sapply(data_imputed[binary_vars], function(x) mean(x == 0) * 100), 2)
)

# Simpan hasil
write.csv(numeric_summary, "./out/summary_numeric_imputed.csv",  quote = FALSE, row.names= FALSE)
write.csv(binary_summary, "./out/summary_binary_imputed.csv",  quote = FALSE, row.names= FALSE)

# Print hasil
print("Ringkasan Statistik Variabel Numerik:")
kable(numeric_summary, row.names = FALSE)

print("Ringkasan Statistik Variabel Kategorik Biner:")
kable(binary_summary, row.names = FALSE)

[1] "Ringkasan Statistik Variabel Numerik:"




|Variable |   N|   Mean|    SD| Median|    Min|    Max|     Q1|     Q3|
|:--------|---:|------:|-----:|------:|------:|------:|------:|------:|
|USIA     | 245|  58.80|  9.41|  60.00|  30.00|  84.00|  53.00|  64.00|
|BB       | 245|  63.65| 10.81|  65.00|  35.00| 105.00|  57.00|  70.00|
|TB       | 245| 160.89|  7.48| 161.00| 141.00| 178.00| 155.00| 165.00|
|TD_S     | 245| 132.82| 24.48| 131.00|  68.00| 201.00| 117.00| 148.00|
|TD_D     | 245|  76.78| 13.15|  75.00|  25.00| 116.00|  68.00|  85.00|
|HR       | 245|  75.28| 16.73|  72.00|  40.00| 130.00|  63.00|  88.00|
|RR       | 245|  22.79|  6.93|  20.00|  15.00|  87.00|  20.00|  28.00|
|HB       | 245|  13.23|  1.76|  13.30|   8.60|  18.20|  12.10|  14.50|
|LEUKOSIT | 245|  11.46| 13.42|   9.57|   2.92| 201.20|   7.64|  12.91|
|PLT      | 245| 252.79| 80.28| 249.00|  22.50| 638.90| 201.90| 301.00|
|BUN      | 245|  17.29|  8.59|  14.95|   7.01|  54.67|  11.68|  20.09|
|SK       | 245|   1.19|  0.78|   1.00|   0.50|  10.00|   0.80

[1] "Ringkasan Statistik Variabel Kategorik Biner:"




|Variable         |   N| Yes_n| Yes_percent| No_n| No_percent|
|:----------------|---:|-----:|-----------:|----:|----------:|
|JENIS_KEL        | 245|   189|       77.14|   56|      22.86|
|RIW_DM           | 245|    63|       25.71|  182|      74.29|
|RIW_HT           | 245|    99|       40.41|  146|      59.59|
|RIW_DISLIPIDEMIA | 245|     3|        1.22|  242|      98.78|
|RIW_MEROKOK      | 245|    56|       22.86|  189|      77.14|
|RIW_KELUARGA     | 245|     1|        0.41|  244|      99.59|
|RIW_ALKOHOL      | 245|     0|        0.00|  245|     100.00|
|HBSAG            | 245|    15|        6.12|  230|      93.88|
|ANTIHCV          | 245|     1|        0.41|  244|      99.59|

#### Visualisasi Data

In [132]:
#4. Visualisasi Sesuai Tipe Data
library(ggplot2)
library(gridExtra)

In [133]:
# a. Visualisasi untuk variabel biner
# Function untuk membuat pie chart
create_piechart <- function(data, var, labels) {
    # Hitung frekuensi dan persentase
    count <- table(data[[var]])
    n_total <- sum(count)
    prop_data <- data.frame(
        category = names(count),
        count = as.numeric(count),
        percentage = as.numeric(prop.table(count) * 100)
    )
    
    # Buat label dengan format n(%)
    prop_data$label <- sprintf("%d (%.1f%%)", 
                             prop_data$count, 
                             prop_data$percentage)
    
    # Ganti nama kategori sesuai labels
    prop_data$category <- labels[prop_data$category]
    
    ggplot(prop_data, aes(x = "", y = percentage, fill = category)) +
        geom_bar(stat = "identity", width = 1) +
        coord_polar("y", start = 0) +
        geom_text(aes(label = label),
                 position = position_stack(vjust = 0.5)) +
        theme_void() +
        labs(title = var,
             fill = "") +
        theme(plot.title = element_text(hjust = 0.5, size = 12, face = "bold"))
}

# Definisi label untuk setiap variabel
labels_list <- list(
    JENIS_KEL = c("0" = "Perempuan", "1" = "Laki-laki"),
    RIW_DM = c("0" = "Tidak", "1" = "Ya"),
    RIW_HT = c("0" = "Tidak", "1" = "Ya"),
    RIW_DISLIPIDEMIA = c("0" = "Tidak", "1" = "Ya"),
    RIW_MEROKOK = c("0" = "Tidak", "1" = "Ya"),
    RIW_KELUARGA = c("0" = "Tidak", "1" = "Ya"),
    RIW_ALKOHOL = c("0" = "Tidak", "1" = "Ya"),
    HBSAG = c("0" = "Non-Reaktif", "1" = "Reaktif"),
    ANTIHCV = c("0" = "Non-Reaktif", "1" = "Reaktif")
)

# Buat pie chart untuk setiap variabel
piecharts <- lapply(names(labels_list), function(var) {
    create_piechart(data_imputed, var, labels_list[[var]])
})

# Simpan plot
png("./out/distribusi_biner_pie.png", 
    width = 1500, 
    height = 1500, 
    res = 200)
do.call(grid.arrange, c(piecharts, ncol = 3))
dev.off()

pdf 
  2

In [134]:
# b. Visualisasi untuk Variabel Numerik
# Fungsi yang sudah dimodifikasi untuk menghasilkan histogram frekuensi
create_histogram <- function(data, var) {
    # Hitung range untuk skala y yang sesuai
    hist_data <- hist(data[[var]], plot = FALSE)
    max_density <- max(hist_data$density) * 1.2
    
    # Hitung breaks untuk sumbu y yang lebih masuk akal
    y_breaks <- pretty(seq(0, max_density, length.out = 5))
    
    ggplot(data, aes_string(x = var)) +
        geom_histogram(aes(y = ..density..), 
                      bins = 30,
                      fill = "#A8D5E5",
                      color = "#2C3E50",
                      alpha = 0.7) +
        geom_density(color = "#E74C3C",
                    linewidth = 1) +
        theme_minimal(base_size = 14) +
        theme(
            plot.title = element_text(size = 16, face = "bold", hjust = 0.5),
            axis.title = element_text(size = 14),
            axis.text = element_text(size = 12),
            plot.margin = margin(t = 15, r = 15, b = 15, l = 15),
            panel.grid.minor = element_blank()
        ) +
        labs(title = paste("Distribusi", var),
             x = var,
             y = "Frekuensi Relatif") + # Mengubah label y menjadi lebih deskriptif
        scale_y_continuous(
            limits = c(0, max_density),
            breaks = y_breaks,
            labels = scales::percent_format(accuracy = 0.1, scale = 100), # Mengubah format ke persentase
            expand = expansion(mult = c(0, 0.05))
        ) +
        scale_x_continuous(
            labels = scales::number_format(accuracy = 1),
            breaks = pretty(data[[var]], n = 5),
            expand = expansion(mult = c(0.05, 0.05))
        )
}

# Contoh variabel
basic_vars <- c("USIA", "BB", "TB")

# Simpan histogram karakteristik dasar menjadi file PNG
png("./out/1_distribusi_karakteristik.png", width = 1800, height = 600, res = 150)
grid.arrange(grobs = lapply(basic_vars, function(var) create_histogram(data_imputed, var)), 
             ncol = 3)
dev.off()

# Variabel lainnya dapat disesuaikan dengan cara yang sama:
vital_vars <- c("TD_S", "TD_D", "HR", "RR")
png("./out/2_distribusi_tanda_vital.png", width = 1800, height = 1000, res = 150)
grid.arrange(grobs = lapply(vital_vars, function(var) create_histogram(data_imputed, var)), 
             ncol = 2)
dev.off()

blood_vars <- c("HB", "LEUKOSIT", "PLT", "BUN")
png("./out/3_distribusi_darah_rutin.png", width = 1800, height = 1000, res = 150)
grid.arrange(grobs = lapply(blood_vars, function(var) create_histogram(data_imputed, var)), 
             ncol = 2)
dev.off()

chem_vars <- c("SK", "NATRIUM", "K", "GDS")
png("./out/4_distribusi_kimia_darah.png", width = 1800, height = 1000, res = 150)
grid.arrange(grobs = lapply(chem_vars, function(var) create_histogram(data_imputed, var)), 
             ncol = 2)
dev.off()

lipid_vars <- c("KOL_TOT", "TG", "HDL", "LDL")
png("./out/5_distribusi_profil_lipid.png", width = 1800, height = 1000, res = 150)
grid.arrange(grobs = lapply(lipid_vars, function(var) create_histogram(data_imputed, var)), 
             ncol = 2)
dev.off()

other_vars <- c("GDP", "UREUM", "HBA1C")
png("./out/6_distribusi_lainnya.png", width = 1800, height = 600, res = 150)
grid.arrange(grobs = lapply(other_vars, function(var) create_histogram(data_imputed, var)), 
             ncol = 3)
dev.off()

png("./out/7_distribusi_gensini.png", width = 800, height = 400, res = 150)
print(create_histogram(data_imputed, "GENSINI"))
dev.off()


Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 3 rows containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 3 rows containing missing values or values outside the scale range
(`geom_bar()`)."


pdf 
  2

Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 2 rows containing missing values or values outside the scale range
(`geom_bar()`)."


pdf 
  2

Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."


...

Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 2 rows containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."


pdf 
  2

Warning message:
"Removed 3 rows containing missing values or values outside the scale range
(`geom_bar()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`)."


pdf 
  2

pdf 
  2

In [135]:
# 1. Membuat data frame untuk analisis segmen
segment_pairs <- data.frame(
    segmen = c("LMCA", "LAD_PROX", "LAD_MID", "LAD_DIST", "LAD_D1", 
               "LAD_D2", "LAD_API", "LCX_PROX", "LCX_DIST", "LCX_OM", 
               "LCX_PL", "RCA_PROX", "RCA_MID", "RCA_DIST", "RCA_PD"),
    point = c("LMCA_POINT", "LAD_PROX_POINT", "LAD_MID_POINT", "LAD_DIST_POINT", 
              "LAD_D1_POINT", "LAD_D2_POINT", "LAD_API_POINT", "LCX_PROX_POINT", 
              "LCX_DIST_POINT", "LCX_OM_POINT", "LCX_PL_POINT", "RCA_PROX_POINT", 
              "RCA_MID_POINT", "RCA_DIST_POINT", "RCA_PD_POINT")
)

# 2. Fungsi untuk menghitung summary statistik segmen
calculate_segment_stats <- function(data, segment_pairs) {
    results <- data.frame()
    
    for(i in 1:nrow(segment_pairs)) {
        seg_name <- segment_pairs$segmen[i]
        point_name <- segment_pairs$point[i]
        
        # Hitung statistik
        n_involved <- sum(data[[point_name]] > 0, na.rm = TRUE)
        mean_stenosis <- round(mean(data[[seg_name]], na.rm = TRUE),2)
        mean_point <- round(mean(data[[point_name]], na.rm = TRUE),2)
        
        results <- rbind(results, data.frame(
            Segment = seg_name,
            N_Involved = n_involved,
            Percent_Involved = round((n_involved/nrow(data))*100,2),
            Mean_Stenosis = mean_stenosis,
            Mean_Point = mean_point
        ))
    }
    
    return(results)
}

# 3. Hitung jumlah segmen terlibat per pasien
calculate_involved_segments <- function(data, segment_pairs) {
    point_cols <- segment_pairs$point
    involved_count <- rowSums(data[point_cols] > 0, na.rm = TRUE)
    mean_involved <- round(mean(involved_count),2)
    sd_involved <- sd(involved_count)
    
    return(data.frame(
        Mean_Segments = mean_involved,
        SD_Segments = round(sd_involved,2),
        Min_Segments = min(involved_count),
        Max_Segments = max(involved_count)
    ))
}

# 4. Lakukan analisis
segment_stats <- calculate_segment_stats(data, segment_pairs)
involvement_stats <- calculate_involved_segments(data, segment_pairs)

# 5. Buat visualisasi
# a. Bar plot untuk N pasien terlibat per segmen
involvement_plot <- ggplot(segment_stats, aes(x = reorder(Segment, -N_Involved), y = N_Involved)) +
    geom_bar(stat = "identity", fill = "#A8D5E5", color = "#2C3E50") +
    theme_minimal(base_size = 14) +
    theme(
        axis.text.x = element_text(angle = 45, hjust = 1),
        plot.title = element_text(size = 16, face = "bold", hjust = 0.5)
    ) +
    labs(title = "Jumlah Pasien Terlibat per Segmen",
         x = "Segmen",
         y = "Jumlah Pasien")

# b. Bar plot untuk rerata stenosis per segmen
stenosis_plot <- ggplot(segment_stats, aes(x = reorder(Segment, -Mean_Stenosis), y = Mean_Stenosis)) +
    geom_bar(stat = "identity", fill = "#A8D5E5", color = "#2C3E50") +
    theme_minimal(base_size = 14) +
    theme(
        axis.text.x = element_text(angle = 45, hjust = 1),
        plot.title = element_text(size = 16, face = "bold", hjust = 0.5)
    ) +
    labs(title = "Rerata Stenosis per Segmen",
         x = "Segmen",
         y = "Rerata Stenosis (%)")

# c. Bar plot untuk rerata point per segmen
point_plot <- ggplot(segment_stats, aes(x = reorder(Segment, -Mean_Point), y = Mean_Point)) +
    geom_bar(stat = "identity", fill = "#A8D5E5", color = "#2C3E50") +
    theme_minimal(base_size = 14) +
    theme(
        axis.text.x = element_text(angle = 45, hjust = 1),
        plot.title = element_text(size = 16, face = "bold", hjust = 0.5)
    ) +
    labs(title = "Rerata Point per Segmen",
         x = "Segmen",
         y = "Rerata Point")

# 6. Simpan hasil
# Simpan statistik ke CSV
write.csv(segment_stats, "./out/summary_segment_statistics.csv", row.names = FALSE, quote = FALSE)
write.csv(involvement_stats, "./out/summary_involvement_statistics.csv", row.names = FALSE, quote = FALSE)

# Simpan plot
png("./out/segment_involvement_plot.png", width = 1800, height = 1000, res = 150)
print(involvement_plot)
dev.off()

png("./out/segment_stenosis_plot.png", width = 1800, height = 1000, res = 150)
print(stenosis_plot)
dev.off()

png("./out/segment_point_plot.png", width = 1800, height = 1000, res = 150)
print(point_plot)
dev.off()

# Print ringkasan
print("Statistik Segmen:")
kable(segment_stats)
print("Statistik Keterlibatan Segmen per Pasien:")
kable(involvement_stats)

pdf 
  2

pdf 
  2

pdf 
  2

[1] "Statistik Segmen:"




|Segment  | N_Involved| Percent_Involved| Mean_Stenosis| Mean_Point|
|:--------|----------:|----------------:|-------------:|----------:|
|LMCA     |         31|            12.65|          7.18|       2.90|
|LAD_PROX |        151|            61.63|         45.73|      17.10|
|LAD_MID  |         84|            34.29|         26.49|       5.82|
|LAD_DIST |         25|            10.20|          7.82|       1.22|
|LAD_D1   |          9|             3.67|          2.89|       0.31|
|LAD_D2   |          0|             0.00|          0.00|       0.00|
|LAD_API  |          0|             0.00|          0.00|       0.00|
|LCX_PROX |         39|            15.92|         12.41|       5.04|
|LCX_DIST |         69|            28.16|         20.28|       2.91|
|LCX_OM   |         20|             8.16|          5.57|       0.45|
|LCX_PL   |          3|             1.22|          0.78|       0.02|
|RCA_PROX |        100|            40.82|         31.16|       5.56|
|RCA_MID  |         72|         

[1] "Statistik Keterlibatan Segmen per Pasien:"




| Mean_Segments| SD_Segments| Min_Segments| Max_Segments|
|-------------:|-----------:|------------:|------------:|
|          2.63|        1.52|            0|           10|

### Analisis Bivariat

In [143]:
# Load required packages
library(Hmisc)
library(corrplot)
library(tidyverse)


In [ ]:
# Fungsi untuk melakukan uji normalitas dan korelasi
perform_correlation_analysis <- function(data, var, target = "GENSINI") {
  # Skip jika variabel tidak memiliki variasi
  if (length(unique(data[[var]])) == 1) {
    return(data.frame(
      Variable = var,
      Method = NA,
      Correlation = NA,
      p_value = NA,
      normality_var = NA,
      normality_target = NA,
      status = "No variation"
    ))
  }
  
  # Uji normalitas
  norm_var <- tryCatch({
    shapiro.test(data[[var]])$p.value
  }, error = function(e) {
    return(0) # Return 0 jika error (tidak normal)
  })
  
  norm_target <- shapiro.test(data[[target]])$p.value
  
  # Pilih metode berdasarkan normalitas
  if(norm_var > 0.05 && norm_target > 0.05) {
    test <- cor.test(data[[var]], data[[target]], 
                     method = "pearson",
                     exact = FALSE)
    method <- "Pearson"
  } else {
    test <- cor.test(data[[var]], data[[target]], 
                     method = "spearman",
                     exact = FALSE)
    method <- "Spearman"
  }
  
  data.frame(
    Variable = var,
    Method = method,
    Correlation = round(as.numeric(test$estimate), 3),
    p_value = round(test$p.value, 4),
    normality_var = round(norm_var, 4),
    normality_target = round(norm_target, 4),
    status = "Analyzed"
  )
}

In [ ]:

# 1. Definisi variabel
numeric_vars <- c("USIA", "BB", "TB", "TD_S", "TD_D", "HR", "RR",
                 "HB", "LEUKOSIT", "PLT", "BUN", "SK", "NATRIUM", "K",
                 "GDS", "GDP", "KOL_TOT", "TG", "HDL", "LDL",
                 "UREUM", "HBA1C")

binary_vars <- c("JENIS_KEL", "RIW_DM", "RIW_HT", "RIW_DISLIPIDEMIA", 
                "RIW_MEROKOK", "HBSAG", "ANTIHCV", "RIW_ALKOHOL" ,"RIW_KELUARGA") 

In [ ]:
# 2. Analisis korelasi
# Untuk variabel numerik
numeric_correlations <- do.call(rbind, lapply(numeric_vars, function(var) {
  perform_correlation_analysis(data_imputed, var)
}))

# Untuk variabel biner
binary_correlations <- do.call(rbind, lapply(binary_vars, function(var) {
  perform_correlation_analysis(data_imputed, var)
}))

In [ ]:
# 3. Gabung dan proses hasil
all_correlations <- rbind(numeric_correlations, binary_correlations) %>%
  mutate(
    abs_corr = abs(Correlation),
    significance = case_when(
      is.na(p_value) ~ "",
      p_value < 0.001 ~ "***",
      p_value < 0.01 ~ "**",
      p_value < 0.05 ~ "*",
      p_value < 0.25 ~ ".",
      TRUE ~ "ns"
    ),
    interpretation = case_when(
      is.na(Correlation) ~ "No variation",
      abs(Correlation) >= 0.7 ~ "Very Strong",
      abs(Correlation) >= 0.5 ~ "Strong",
      abs(Correlation) >= 0.3 ~ "Moderate",
      abs(Correlation) >= 0.1 ~ "Weak",
      TRUE ~ "Very Weak"
    ),
    eligible_for_model = !is.na(p_value) & p_value < 0.25
  ) %>%
  arrange(desc(abs_corr))

# 4. Simpan hasil
write.csv(all_correlations, "./out/biv_all_correlations.csv", row.names = FALSE, quote = FALSE)

# 5. Visualisasi untuk variabel yang eligible
eligible_vars <- all_correlations %>%
  filter(eligible_for_model) %>%
  pull(Variable)

if(length(eligible_vars) > 0) {
  # Matrix korelasi
  correlation_matrix <- cor(data_imputed[c(eligible_vars, "GENSINI")], 
                          method = "spearman")
  
  png("./out/biv_correlation_matrix.png", width = 2000, height = 2000, res = 200)
  corrplot(correlation_matrix,
           method = "color",
           type = "upper",
           order = "hclust",
           addCoef.col = "black",
           tl.col = "black",
           tl.srt = 45,
           number.cex = 0.7,
           tl.cex = 0.7,
           diag = FALSE)
  dev.off()
  
  # Scatter plots untuk variabel numerik yang eligible
  eligible_numeric_vars <- intersect(eligible_vars, numeric_vars)
  
  for(var in eligible_numeric_vars) {
    cor_info <- all_correlations[all_correlations$Variable == var, ]
    
    plot <- ggplot(data_imputed, aes_string(x = var, y = "GENSINI")) +
      geom_point(alpha = 0.5) +
      geom_smooth(method = "lm", color = "red") +
      theme_minimal() +
      labs(
        title = paste("Scatter Plot:", var, "vs Gensini Score"),
        subtitle = paste0(
          cor_info$Method, " correlation: ", round(cor_info$Correlation, 3),
          " (", cor_info$interpretation, ")\n",
          "p-value: ", format.pval(cor_info$p_value, digits = 3),
          cor_info$significance
        )
      )
    
    ggsave(paste0("./out/biv_scatter_", var, ".png"), 
           plot, width = 8, height = 6)
  }
  
  # Box plots untuk variabel biner yang eligible
  eligible_binary_vars <- intersect(eligible_vars, binary_vars)
  
  for(var in eligible_binary_vars) {
    cor_info <- all_correlations[all_correlations$Variable == var, ]
    
    plot <- ggplot(data_imputed, aes_string(x = var, y = "GENSINI")) +
      geom_boxplot() +
      theme_minimal() +
      labs(
        title = paste("Box Plot:", var, "vs Gensini Score"),
        subtitle = paste0(
          cor_info$Method, " correlation: ", round(cor_info$Correlation, 3),
          " (", cor_info$interpretation, ")\n",
          "p-value: ", format.pval(cor_info$p_value, digits = 3),
          cor_info$significance
        )
      )
    
    ggsave(paste0("./out/biv_boxplot_", var, ".png"), 
           plot, width = 8, height = 6)
  }
}

# Print ringkasan
cat("\nRingkasan Analisis Korelasi:\n")
cat("============================\n")
cat("Total variabel dianalisis:", nrow(all_correlations), "\n")
cat("Variabel eligible (p < 0.25):", sum(all_correlations$eligible_for_model), "\n")
cat("Variabel tanpa variasi:", sum(all_correlations$status == "No variation"), "\n\n")

cat("Variabel eligible untuk model (p < 0.25):\n")
print(all_correlations %>% 
  filter(eligible_for_model) %>%
  select(Variable, Method, Correlation, p_value, interpretation, significance))

### Analisis VIF

In [149]:
# Load required packages
library(car)
library(tidyverse)

In [151]:
# Pilih variabel untuk analisis VIF (11 dari p < 0.25 + 3 pertimbangan klinis)
selected_vars <- c(
    # Variabel dengan p < 0.25
    "RR", "TD_S", "USIA", "TG", "HDL", "RIW_HT",
    "LEUKOSIT", "UREUM", "GDS", "BB", "SK",
    
    # Variabel tambahan dari pertimbangan klinis
    "HBA1C", "KOL_TOT", "LDL"
)

# Buat dataset dengan variabel terpilih
vif_data <- data_imputed[selected_vars]

# Lakukan analisis VIF
vif_results <- data.frame(
    Variable = names(vif_data),
    VIF = round(vif(lm(GENSINI ~ ., data = cbind(vif_data, GENSINI = data_imputed$GENSINI))), 2)
) %>%
    arrange(desc(VIF))

# Tambahkan interpretasi
vif_results <- vif_results %>%
    mutate(
        Multicollinearity = case_when(
            VIF > 10 ~ "Serius",
            VIF > 5 ~ "Moderat",
            VIF > 2.5 ~ "Ringan",
            TRUE ~ "Minimal"
        )
    )

# Simpan hasil
write.csv(vif_results, "./out/vif_results_01.csv", row.names = FALSE, quote = FALSE)

# Print hasil
print("Hasil Analisis VIF:")
print(vif_results)

# Identifikasi variabel dengan VIF tinggi
high_vif <- vif_results %>%
    filter(VIF > 5) %>%
    arrange(desc(VIF))

if(nrow(high_vif) > 0) {
    cat("\nVariabel dengan multikolinearitas moderat-serius (VIF > 5):\n")
    print(high_vif)
}

[1] "Hasil Analisis VIF:"
         Variable  VIF Multicollinearity
KOL_TOT   KOL_TOT 5.76           Moderat
LDL           LDL 4.40            Ringan
RR             RR 1.86           Minimal
TG             TG 1.86           Minimal
UREUM       UREUM 1.65           Minimal
HDL           HDL 1.62           Minimal
SK             SK 1.43           Minimal
TD_S         TD_S 1.40           Minimal
BB             BB 1.36           Minimal
RIW_HT     RIW_HT 1.29           Minimal
USIA         USIA 1.22           Minimal
GDS           GDS 1.19           Minimal
LEUKOSIT LEUKOSIT 1.08           Minimal
HBA1C       HBA1C 1.08           Minimal

Variabel dengan multikolinearitas moderat-serius (VIF > 5):
        Variable  VIF Multicollinearity
KOL_TOT  KOL_TOT 5.76           Moderat


In [203]:
# Pilih variabel untuk analisis VIF (11 dari p < 0.25 + 3 pertimbangan klinis)
# Eksklusi KOL_TOT
selected_vars <- c(
    # Variabel dengan p < 0.25
    "RR", "TD_S", "USIA", "TG", "HDL", "RIW_HT",
    "LEUKOSIT", "UREUM", "GDS", "BB", "SK",
    
    # Variabel tambahan dari pertimbangan klinis
    "HBA1C", "LDL"
)

# Buat dataset dengan variabel terpilih
vif_data <- data_imputed[selected_vars]

# Lakukan analisis VIF
vif_results <- data.frame(
    Variable = names(vif_data),
    VIF = round(vif(lm(GENSINI ~ ., data = cbind(vif_data, GENSINI = data_imputed$GENSINI))), 2)
) %>%
    arrange(desc(VIF))

# Tambahkan interpretasi
vif_results <- vif_results %>%
    mutate(
        Multicollinearity = case_when(
            VIF > 10 ~ "Serius",
            VIF > 5 ~ "Moderat",
            VIF > 2.5 ~ "Ringan",
            TRUE ~ "Minimal"
        )
    )

# Simpan hasil
write.csv(vif_results, "./out/vif_results_02.csv", row.names = FALSE, quote = FALSE)

# Print hasil
print("Hasil Analisis VIF:")
print(vif_results)

# Identifikasi variabel dengan VIF tinggi
high_vif <- vif_results %>%
    filter(VIF > 5) %>%
    arrange(desc(VIF))

if(nrow(high_vif) > 0) {
    cat("\nVariabel dengan multikolinearitas moderat-serius (VIF > 5):\n")
    print(high_vif)
}

[1] "Hasil Analisis VIF:"


         Variable  VIF Multicollinearity
RR             RR 1.79           Minimal
UREUM       UREUM 1.64           Minimal
LDL           LDL 1.62           Minimal
HDL           HDL 1.59           Minimal
SK             SK 1.40           Minimal
TD_S         TD_S 1.32           Minimal
RIW_HT     RIW_HT 1.29           Minimal
USIA         USIA 1.22           Minimal
BB             BB 1.19           Minimal
TG             TG 1.15           Minimal
GDS           GDS 1.14           Minimal
LEUKOSIT LEUKOSIT 1.07           Minimal
HBA1C       HBA1C 1.05           Minimal


In [ ]:
write.csv(head(data), "./out/raw_data_summary.csv", row.names = FALSE, quote = FALSE)

### Pengembangan Model

In [194]:
# Load required packages
library(rms)      # untuk OLS dan restricted cubic splines
library(mgcv)     # untuk GAM
library(glmnet)   # untuk ridge regression
library(splines2) # untuk fungsi basis spline tambahan
library(ggplot2)  # untuk visualisasi
library(gridExtra)# untuk menyusun multiple plot

In [195]:
###### 1. PERSIAPAN DATA ######
# Tambahkan skor Gensini, RIW_HT, dan USIA ke setiap dataset hasil imputasi
for(i in 1:length(imputed_datasets)) {
  imputed_datasets[[i]]$GENSINI <- data$GENSINI
  imputed_datasets[[i]]$RIW_HT <- data$RIW_HT
  imputed_datasets[[i]]$USIA <- data$USIA
}

# Verifikasi data sudah lengkap
print("Mengecek kelengkapan data:")
print(paste("Jumlah dataset imputasi:", length(imputed_datasets)))
print(paste("Dimensi dataset pertama:", paste(dim(imputed_datasets[[1]]), collapse="x")))
print("Kolom dalam dataset pertama:")
print(names(imputed_datasets[[1]]))

[1] "Mengecek kelengkapan data:"
[1] "Jumlah dataset imputasi: 10"
[1] "Dimensi dataset pertama: 245x14"
[1] "Kolom dalam dataset pertama:"
 [1] "RR"       "UREUM"    "LDL"      "HDL"      "SK"       "TD_S"    
 [7] "RIW_HT"   "USIA"     "BB"       "TG"       "GDS"      "LEUKOSIT"
[13] "HBA1C"    "GENSINI" 


#### Model OLS

In [196]:
###### 2. MODEL OLS ######
# List untuk menyimpan model OLS dari setiap imputasi
ols_models <- list()

# Fit model untuk setiap dataset hasil imputasi
for(i in 1:length(imputed_datasets)) {
  # Formula OLS
  ols_models[[i]] <- ols(
    GENSINI ~ rcs(RR,3) + rcs(UREUM,4) + rcs(LDL,3) + rcs(HDL,3) + 
              rcs(SK,4) + rcs(TD_S,3) + RIW_HT + USIA + rcs(BB,3) + 
              rcs(TG,4) + rcs(GDS,3) + rcs(LEUKOSIT,4) + HBA1C,
    data = imputed_datasets[[i]],
    x = TRUE,  # simpan matriks x untuk prediksi
    y = TRUE   # simpan vektor y untuk validasi
  )
  print(paste("Model OLS ke-", i, "selesai dibuat"))
}


[1] "Model OLS ke- 1 selesai dibuat"
[1] "Model OLS ke- 2 selesai dibuat"
[1] "Model OLS ke- 3 selesai dibuat"
[1] "Model OLS ke- 4 selesai dibuat"
[1] "Model OLS ke- 5 selesai dibuat"
[1] "Model OLS ke- 6 selesai dibuat"
[1] "Model OLS ke- 7 selesai dibuat"
[1] "Model OLS ke- 8 selesai dibuat"
[1] "Model OLS ke- 9 selesai dibuat"
[1] "Model OLS ke- 10 selesai dibuat"


In [197]:
###### 3. FUNGSI PREDIKSI OLS ######
predict_ols <- function(newdata, models) {
  # Prediksi untuk setiap model
  predictions <- sapply(models, function(model) {
    predict(model, newdata = newdata)
  })
  
  # Rata-rata prediksi dari semua model
  rowMeans(predictions)
}

In [198]:
# 1. Periksa struktur imputed_datasets
print("Struktur imputed_datasets:")
print(class(imputed_datasets))
print(length(imputed_datasets))

# 2. Periksa struktur dataset pertama
print("\nStruktur dataset pertama:")
print(class(imputed_datasets[[1]]))
print(dim(imputed_datasets[[1]]))
print("\nNama kolom dataset pertama:")
print(names(imputed_datasets[[1]]))

# 3. Coba akses beberapa nilai
print("\nCoba akses nilai:")
print("Baris pertama dataset pertama:")
print(head(imputed_datasets[[1]], 1))

[1] "Struktur imputed_datasets:"
[1] "list"
[1] 10
[1] "\nStruktur dataset pertama:"
[1] "data.frame"


[1] 245  14
[1] "\nNama kolom dataset pertama:"
 [1] "RR"       "UREUM"    "LDL"      "HDL"      "SK"       "TD_S"    
 [7] "RIW_HT"   "USIA"     "BB"       "TG"       "GDS"      "LEUKOSIT"
[13] "HBA1C"    "GENSINI" 
[1] "\nCoba akses nilai:"
[1] "Baris pertama dataset pertama:"
  RR UREUM LDL HDL  SK TD_S RIW_HT USIA BB TG GDS LEUKOSIT HBA1C GENSINI
1 20    26  45  43 0.8  153      0   83 65 77 135    8.286   5.6      14


In [199]:
# Konversi hasil imputasi ke format data frame
imputed_dfs <- list()
for(i in 1:length(imputed_datasets)) {
    # Buat data frame dari list kolom
    imp_df <- data.frame(
        RR = imputed_datasets[[i]]$RR,
        UREUM = imputed_datasets[[i]]$UREUM,
        LDL = imputed_datasets[[i]]$LDL,
        HDL = imputed_datasets[[i]]$HDL,
        SK = imputed_datasets[[i]]$SK,
        TD_S = imputed_datasets[[i]]$TD_S,
        RIW_HT = imputed_datasets[[i]]$RIW_HT,
        USIA = imputed_datasets[[i]]$USIA,
        BB = imputed_datasets[[i]]$BB,
        TG = imputed_datasets[[i]]$TG,
        GDS = imputed_datasets[[i]]$GDS,
        LEUKOSIT = imputed_datasets[[i]]$LEUKOSIT,
        HBA1C = imputed_datasets[[i]]$HBA1C,
        GENSINI = imputed_datasets[[i]]$GENSINI
    )
    imputed_dfs[[i]] <- imp_df
}

# Verifikasi struktur data baru
print("Struktur data frame hasil konversi:")
print(class(imputed_dfs[[1]]))
print(dim(imputed_dfs[[1]]))
print(names(imputed_dfs[[1]]))

# Jika verifikasi berhasil, update imputed_datasets
imputed_datasets <- imputed_dfs

[1] "Struktur data frame hasil konversi:"
[1] "data.frame"
[1] 245  14
 [1] "RR"       "UREUM"    "LDL"      "HDL"      "SK"       "TD_S"    
 [7] "RIW_HT"   "USIA"     "BB"       "TG"       "GDS"      "LEUKOSIT"
[13] "HBA1C"    "GENSINI" 


In [200]:
###### 4. VALIDASI BOOTSTRAP UNTUK OLS ######
n_bootstrap <- 10
bootstrap_performance_ols <- matrix(NA, n_bootstrap, 4)  # Tambah 1 kolom untuk RMSE
colnames(bootstrap_performance_ols) <- c("MAE", "MSE", "RMSE", "R2")

for(b in 1:n_bootstrap) {
  # Bootstrap sample
  boot_index <- sample(nrow(imputed_datasets[[1]]), replace = TRUE)
  
  # Fit model pada bootstrap sample dengan as.formula()
  boot_models <- lapply(imputed_datasets, function(data) {
    boot_data <- data[boot_index, ]
    
    # Gunakan as.formula() untuk menghindari warning
    formula_ols <- as.formula(
      "GENSINI ~ rcs(RR,3) + rcs(UREUM,4) + rcs(LDL,3) + rcs(HDL,3) + 
                rcs(SK,4) + rcs(TD_S,3) + RIW_HT + USIA + rcs(BB,3) + 
                rcs(TG,4) + rcs(GDS,3) + rcs(LEUKOSIT,4) + HBA1C"
    )
    
    ols(formula_ols, data = boot_data)
  })
  
  # Prediksi pada data original
  pred <- predict_ols(imputed_datasets[[1]], boot_models)
  obs <- imputed_datasets[[1]]$GENSINI
  
  # Hitung metrik performa
  bootstrap_performance_ols[b, "MAE"] <- round(mean(abs(pred - obs)),2)
  bootstrap_performance_ols[b, "MSE"] <- round(mean((pred - obs)^2),2)
  bootstrap_performance_ols[b, "RMSE"] <- round(sqrt(bootstrap_performance_ols[b, "MSE"]),2)
  bootstrap_performance_ols[b, "R2"] <- round(1 - sum((obs - pred)^2) / sum((obs - mean(obs))^2),2)
  
  print(paste("Bootstrap iteration", b, "completed"))
}

# Optimism-corrected performance metrics
optimism_ols <- colMeans(bootstrap_performance_ols)
print("\nOptimism-corrected performance metrics:")
print(optimism_ols)

# Print detail performa per iterasi dengan format yang lebih rapi
print("\nPerforma per iterasi bootstrap:")
print(round(bootstrap_performance_ols, 3))

# Simpan hasil performa OLS
write.csv(bootstrap_performance_ols, "./out/performance__ols_bootstrap_performance.csv", 
          row.names = FALSE, quote = FALSE)
write.csv(t(as.matrix(optimism_ols)), "./out/performance__ols_optimism_corrected.csv", 
          row.names = FALSE, quote = FALSE)

Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.cha

[1] "Bootstrap iteration 1 completed"


Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.cha

[1] "Bootstrap iteration 2 completed"


Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.cha

...

[1] "Bootstrap iteration 8 completed"


Warning message in rcspline.eval(x, nk = nknots, inclx = TRUE, pc = pc, fractied = fractied):
"could not obtain 3 interior knots with default algorithm.
 Used alternate algorithm to obtain 3 knots"
Warning message in rcspline.eval(x, nk = nknots, inclx = TRUE, pc = pc, fractied = fractied):
"could not obtain 3 interior knots with default algorithm.
 Used alternate algorithm to obtain 3 knots"
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in for

[1] "Bootstrap iteration 9 completed"


Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.cha

[1] "Bootstrap iteration 10 completed"
[1] "\nOptimism-corrected performance metrics:"
     MAE      MSE     RMSE       R2 
  30.057 1959.583   42.817   -0.281 
[1] "\nPerforma per iterasi bootstrap:"
        MAE     MSE  RMSE    R2
 [1,] 31.27 2171.19 46.60 -0.42
 [2,] 30.83 1552.16 39.40 -0.01
 [3,] 34.14 5722.59 75.65 -2.74
 [4,] 28.61 1421.36 37.70  0.07
 [5,] 28.85 1415.91 37.63  0.07
 [6,] 28.84 1448.93 38.06  0.05
 [7,] 29.01 1397.17 37.38  0.09
 [8,] 30.24 1497.00 38.69  0.02
 [9,] 29.52 1440.09 37.95  0.06
[10,] 29.26 1529.43 39.11  0.00


#### Model GAM

In [208]:
###### 5. MODEL GAM ######
# List untuk menyimpan model GAM dari setiap imputasi
gam_models <- list()

# Fit model untuk setiap dataset hasil imputasi
for(i in 1:length(imputed_datasets)) {
  # Formula GAM dengan penalized smoothing splines
  formula_gam <- as.formula(
    "GENSINI ~ s(RR) + s(UREUM, k=4) + s(LDL) + s(HDL) + 
              s(SK, k=4) + s(TD_S) + RIW_HT + USIA + s(BB) + 
              s(TG, k=4) + s(GDS) + s(LEUKOSIT, k=4) + s(HBA1C)"
  )
  
  gam_models[[i]] <- gam(formula_gam, data = imputed_datasets[[i]])
  print(paste("Model GAM ke-", i, "selesai dibuat"))
}


[1] "Model GAM ke- 1 selesai dibuat"
[1] "Model GAM ke- 2 selesai dibuat"
[1] "Model GAM ke- 3 selesai dibuat"
[1] "Model GAM ke- 4 selesai dibuat"
[1] "Model GAM ke- 5 selesai dibuat"
[1] "Model GAM ke- 6 selesai dibuat"
[1] "Model GAM ke- 7 selesai dibuat"
[1] "Model GAM ke- 8 selesai dibuat"
[1] "Model GAM ke- 9 selesai dibuat"
[1] "Model GAM ke- 10 selesai dibuat"


In [209]:
###### 6. FUNGSI PREDIKSI GAM ######
predict_gam <- function(newdata, models) {
  # Prediksi untuk setiap model
  predictions <- sapply(models, function(model) {
    predict(model, newdata = newdata)
  })
  
  # Rata-rata prediksi dari semua model
  rowMeans(predictions)
}

In [210]:
###### 7. VALIDASI BOOTSTRAP UNTUK GAM ######
n_bootstrap <- 10
bootstrap_performance_gam <- matrix(NA, n_bootstrap, 4)
colnames(bootstrap_performance_gam) <- c("MAE", "MSE", "RMSE", "R2")

for(b in 1:n_bootstrap) {
  # Bootstrap sample
  boot_index <- sample(nrow(imputed_datasets[[1]]), replace = TRUE)
  
  # Fit model pada bootstrap sample
  boot_models <- lapply(imputed_datasets, function(data) {
    boot_data <- data[boot_index, ]
    gam(formula_gam, data = boot_data)
  })
  
  # Prediksi pada data original
  pred <- predict_gam(imputed_datasets[[1]], boot_models)
  obs <- imputed_datasets[[1]]$GENSINI
  
  # Hitung metrik performa
  bootstrap_performance_gam[b, "MAE"] <- round(mean(abs(pred - obs)),2)
  bootstrap_performance_gam[b, "MSE"] <- round(mean((pred - obs)^2))
  bootstrap_performance_gam[b, "RMSE"] <- round(sqrt(bootstrap_performance_gam[b, "MSE"]))
  bootstrap_performance_gam[b, "R2"] <- round(1 - sum((obs - pred)^2) / sum((obs - mean(obs))^2))
  
  print(paste("Bootstrap iteration", b, "completed"))
}

# Optimism-corrected performance metrics
optimism_gam <- colMeans(bootstrap_performance_gam)
print("\nOptimism-corrected performance metrics GAM:")
print(optimism_gam)

# Print detail performa per iterasi
print("\nPerforma per iterasi bootstrap GAM:")
print(round(bootstrap_performance_gam, 3))

# Simpan hasil performa GAM
write.csv(bootstrap_performance_gam, "./out/performance__gam_bootstrap_performance.csv", 
          row.names = FALSE, quote = FALSE)
write.csv(t(as.matrix(optimism_gam)), "./out/performance__gam_optimism_corrected.csv", 
          row.names = FALSE, quote = FALSE)

[1] "Bootstrap iteration 1 completed"
[1] "Bootstrap iteration 2 completed"
[1] "Bootstrap iteration 3 completed"
[1] "Bootstrap iteration 4 completed"
[1] "Bootstrap iteration 5 completed"
[1] "Bootstrap iteration 6 completed"
[1] "Bootstrap iteration 7 completed"
[1] "Bootstrap iteration 8 completed"
[1] "Bootstrap iteration 9 completed"
[1] "Bootstrap iteration 10 completed"
[1] "\nOptimism-corrected performance metrics GAM:"
     MAE      MSE     RMSE       R2 
  29.049 1591.300   39.800   -0.100 
[1] "\nPerforma per iterasi bootstrap GAM:"
        MAE  MSE RMSE R2
 [1,] 29.12 1831   43  0
 [2,] 31.30 2301   48 -1
 [3,] 28.38 1412   38  0
 [4,] 31.60 1931   44  0
 [5,] 28.21 1387   37  0
 [6,] 29.30 1502   39  0
 [7,] 27.94 1410   38  0
 [8,] 27.89 1295   36  0
 [9,] 27.81 1447   38  0
[10,] 28.94 1397   37  0


#### Model RIDGE

In [211]:
###### 8. MODEL RIDGE REGRESSION ######
library(glmnet)


In [212]:
# Fungsi untuk membuat matriks design dengan splines
create_design_matrix <- function(data) {
  # Inisialisasi list untuk menyimpan basis splines
  spline_bases <- list()
  
  # Buat basis splines untuk setiap variabel
  spline_bases$RR <- rcs(data$RR, 3)
  spline_bases$UREUM <- rcs(data$UREUM, 4)
  spline_bases$LDL <- rcs(data$LDL, 3)
  spline_bases$HDL <- rcs(data$HDL, 3)
  spline_bases$SK <- rcs(data$SK, 4)
  spline_bases$TD_S <- rcs(data$TD_S, 3)
  spline_bases$BB <- rcs(data$BB, 3)
  spline_bases$TG <- rcs(data$TG, 4)
  spline_bases$GDS <- rcs(data$GDS, 3)
  spline_bases$LEUKOSIT <- rcs(data$LEUKOSIT, 4)
  
  # Gabungkan dalam satu matriks
  X <- do.call(cbind, c(
    lapply(spline_bases, as.matrix),
    list(
      RIW_HT = data$RIW_HT,
      USIA = data$USIA,
      HBA1C = data$HBA1C
    )
  ))
  
  return(as.matrix(X))
}

In [213]:
# List untuk menyimpan model ridge
ridge_models <- list()

# Grid nilai lambda untuk CV
lambdas <- 10^seq(2, -10, by = -0.5)

# Fit model untuk setiap dataset hasil imputasi
for(i in 1:length(imputed_datasets)) {
  # Buat matriks design
  X <- create_design_matrix(imputed_datasets[[i]])
  y <- imputed_datasets[[i]]$GENSINI
  
  # Cross-validation untuk memilih lambda optimal
  cv_ridge <- cv.glmnet(X, y, alpha = 0, lambda = lambdas)
  
  # Fit model dengan lambda optimal
  ridge_models[[i]] <- glmnet(X, y, alpha = 0, lambda = cv_ridge$lambda.min)
  
  print(paste("Model Ridge ke-", i, "selesai dibuat"))
}

[1] "Model Ridge ke- 1 selesai dibuat"
[1] "Model Ridge ke- 2 selesai dibuat"
[1] "Model Ridge ke- 3 selesai dibuat"
[1] "Model Ridge ke- 4 selesai dibuat"
[1] "Model Ridge ke- 5 selesai dibuat"
[1] "Model Ridge ke- 6 selesai dibuat"
[1] "Model Ridge ke- 7 selesai dibuat"
[1] "Model Ridge ke- 8 selesai dibuat"
[1] "Model Ridge ke- 9 selesai dibuat"
[1] "Model Ridge ke- 10 selesai dibuat"


In [215]:
###### 9. FUNGSI PREDIKSI RIDGE ######
predict_ridge <- function(newdata, models) {
  # Buat design matrix untuk data baru
  X_new <- create_design_matrix(newdata)
  
  # Prediksi untuk setiap model
  predictions <- sapply(models, function(model) {
    predict(model, newx = X_new)
  })
  
  # Rata-rata prediksi dari semua model
  rowMeans(predictions)
}

In [217]:
###### 10. VALIDASI BOOTSTRAP UNTUK RIDGE ######
n_bootstrap <- 10
bootstrap_performance_ridge <- matrix(NA, n_bootstrap, 4)
colnames(bootstrap_performance_ridge) <- c("MAE", "MSE", "RMSE", "R2")

for(b in 1:n_bootstrap) {
  # Bootstrap sample
  boot_index <- sample(nrow(imputed_datasets[[1]]), replace = TRUE)
  
  # Fit model pada bootstrap sample
  boot_models <- lapply(imputed_datasets, function(data) {
    boot_data <- data[boot_index, ]
    X <- create_design_matrix(boot_data)
    y <- boot_data$GENSINI
    
    # Cross-validation untuk lambda optimal
    cv_ridge <- cv.glmnet(X, y, alpha = 0, lambda = lambdas)
    
    # Fit model
    glmnet(X, y, alpha = 0, lambda = cv_ridge$lambda.min)
  })
  
  # Prediksi pada data original
  pred <- predict_ridge(imputed_datasets[[1]], boot_models)
  obs <- imputed_datasets[[1]]$GENSINI
  
  # Hitung metrik performa
  bootstrap_performance_ridge[b, "MAE"] <- round(mean(abs(pred - obs)),2)
  bootstrap_performance_ridge[b, "MSE"] <- round(mean((pred - obs)^2))
  bootstrap_performance_ridge[b, "RMSE"] <- round(sqrt(bootstrap_performance_ridge[b, "MSE"]),2)
  bootstrap_performance_ridge[b, "R2"] <- 1 - round(sum((obs - pred)^2) / sum((obs - mean(obs))^2),2)
  
  print(paste("Bootstrap iteration", b, "completed"))
}

# Optimism-corrected performance metrics
optimism_ridge <- colMeans(bootstrap_performance_ridge)
print("\nOptimism-corrected performance metrics Ridge:")
print(optimism_ridge)

# Print detail performa per iterasi
print("\nPerforma per iterasi bootstrap Ridge:")
print(round(bootstrap_performance_ridge, 3))

# Simpan hasil performa Ridge
write.csv(bootstrap_performance_ridge, "./out/performance__ridge_bootstrap_performance.csv", 
          row.names = FALSE, quote = FALSE)
write.csv(t(as.matrix(optimism_ridge)), "./out/performance__ridge_optimism_corrected.csv", 
          row.names = FALSE, quote = FALSE)

[1] "Bootstrap iteration 1 completed"
[1] "Bootstrap iteration 2 completed"
[1] "Bootstrap iteration 3 completed"
[1] "Bootstrap iteration 4 completed"
[1] "Bootstrap iteration 5 completed"
[1] "Bootstrap iteration 6 completed"
[1] "Bootstrap iteration 7 completed"
[1] "Bootstrap iteration 8 completed"
[1] "Bootstrap iteration 9 completed"
[1] "Bootstrap iteration 10 completed"
[1] "\nOptimism-corrected performance metrics Ridge:"
     MAE      MSE     RMSE       R2 
  31.086 1627.900   40.005   -0.063 
[1] "\nPerforma per iterasi bootstrap Ridge:"
        MAE  MSE  RMSE    R2
 [1,] 31.82 1581 39.76 -0.03
 [2,] 29.77 1417 37.64  0.07
 [3,] 29.35 1395 37.35  0.09
 [4,] 30.26 1461 38.22  0.05
 [5,] 28.98 1416 37.63  0.07
 [6,] 30.62 1468 38.31  0.04
 [7,] 30.07 1425 37.75  0.07
 [8,] 38.89 3083 55.52 -1.01
 [9,] 29.60 1457 38.17  0.05
[10,] 31.50 1576 39.70 -0.03


#### Perbandingan Performa Model

In [218]:
# Gabungkan semua hasil performa
all_performance <- data.frame(
  Model = rep(c("OLS", "GAM", "Ridge"), each = n_bootstrap),
  MAE = c(bootstrap_performance_ols[,"MAE"], 
          bootstrap_performance_gam[,"MAE"], 
          bootstrap_performance_ridge[,"MAE"]),
  MSE = c(bootstrap_performance_ols[,"MSE"], 
          bootstrap_performance_gam[,"MSE"], 
          bootstrap_performance_ridge[,"MSE"]),
  RMSE = c(bootstrap_performance_ols[,"RMSE"], 
           bootstrap_performance_gam[,"RMSE"], 
           bootstrap_performance_ridge[,"RMSE"]),
  R2 = c(bootstrap_performance_ols[,"R2"], 
         bootstrap_performance_gam[,"R2"], 
         bootstrap_performance_ridge[,"R2"])
)


In [220]:
# Buat boxplot untuk setiap metrik
library(ggplot2)
library(gridExtra)

p1 <- ggplot(all_performance, aes(x = Model, y = MAE, fill = Model)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "MAE Across Models", y = "Mean Absolute Error")

p2 <- ggplot(all_performance, aes(x = Model, y = RMSE, fill = Model)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "RMSE Across Models", y = "Root Mean Square Error")

p3 <- ggplot(all_performance, aes(x = Model, y = R2, fill = Model)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "R² Across Models", y = "R-squared")

# Simpan plot
ggsave("./out/performance__model_comparison_plots.png", 
       arrangeGrob(p1, p2, p3, ncol = 3),
       width = 15, height = 5)

### Kalibrasi Model

In [221]:
###### 11. KALIBRASI MODEL ######
# Fungsi untuk menghitung kalibrasi
calculate_calibration <- function(observed, predicted) {
  # Fit regresi linear observed vs predicted
  cal_model <- lm(observed ~ predicted)
  
  # Hitung statistik kalibrasi
  calibration <- list(
    intercept = coef(cal_model)[1],
    slope = coef(cal_model)[2],
    r_squared = summary(cal_model)$r.squared,
    calibration_in_large = mean(observed) - mean(predicted)
  )
  
  return(calibration)
}


In [222]:
# Hitung kalibrasi untuk setiap model
calibration_results <- list()

# OLS
pred_ols <- predict_ols(imputed_datasets[[1]], ols_models)
calibration_results$OLS <- calculate_calibration(imputed_datasets[[1]]$GENSINI, pred_ols)

# GAM
pred_gam <- predict_gam(imputed_datasets[[1]], gam_models)
calibration_results$GAM <- calculate_calibration(imputed_datasets[[1]]$GENSINI, pred_gam)

# Ridge
pred_ridge <- predict_ridge(imputed_datasets[[1]], ridge_models)
calibration_results$Ridge <- calculate_calibration(imputed_datasets[[1]]$GENSINI, pred_ridge)

# Konversi hasil ke data frame
calibration_df <- data.frame(
  Model = names(calibration_results),
  Intercept = sapply(calibration_results, function(x) x$intercept),
  Slope = sapply(calibration_results, function(x) x$slope),
  R_squared = sapply(calibration_results, function(x) x$r_squared),
  Calibration_in_large = sapply(calibration_results, function(x) x$calibration_in_large)
)

# Simpan hasil kalibrasi
write.csv(calibration_df, "./out/performance__calibration_results.csv", 
          row.names = FALSE, quote = FALSE)

# Visualisasi kalibrasi
# Plot kalibrasi untuk setiap model
plot_calibration <- function(observed, predicted, model_name) {
  data <- data.frame(observed = observed, predicted = predicted)
  
  ggplot(data, aes(x = predicted, y = observed)) +
    geom_point(alpha = 0.5) +
    geom_smooth(method = "lm", col = "red") +
    geom_abline(intercept = 0, slope = 1, linetype = "dashed") +
    theme_minimal() +
    labs(title = paste("Calibration Plot -", model_name),
         x = "Predicted Gensini Score",
         y = "Observed Gensini Score")
}

p1 <- plot_calibration(imputed_datasets[[1]]$GENSINI, pred_ols, "OLS")
p2 <- plot_calibration(imputed_datasets[[1]]$GENSINI, pred_gam, "GAM")
p3 <- plot_calibration(imputed_datasets[[1]]$GENSINI, pred_ridge, "Ridge")

# Simpan plot kalibrasi
ggsave("./out/performance__calibration_plots.png", 
       arrangeGrob(p1, p2, p3, ncol = 3),
       width = 15, height = 5)

print("Hasil Kalibrasi:")
print(calibration_df)

Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.character(object, env = baseenv()):
"Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead."
Warning message in formula.cha

[1] "Hasil Kalibrasi:"
                  Model  Intercept    Slope R_squared Calibration_in_large
OLS.(Intercept)     OLS  -1.132307 1.025630 0.1487694           0.08250277
GAM.(Intercept)     GAM  -9.082098 1.191949 0.1622756           0.02649749
Ridge.(Intercept) Ridge -16.441178 1.388715 0.1053652           1.45088984


### Ekstraksi Koefisien OLS

In [300]:
# Tampilkan nama koefisien dari model pertama
print(names(coef(ols_models[[1]])))


 [1] "Intercept"  "RR"         "RR'"        "UREUM"      "UREUM'"    
 [6] "UREUM''"    "LDL"        "LDL'"       "HDL"        "HDL'"      
[11] "SK"         "SK'"        "SK''"       "TD_S"       "TD_S'"     
[16] "RIW_HT"     "USIA"       "BB"         "BB'"        "TG"        
[21] "TG'"        "TG''"       "GDS"        "GDS'"       "LEUKOSIT"  
[26] "LEUKOSIT'"  "LEUKOSIT''" "HBA1C"     


In [301]:
library(jsonlite)

In [302]:
extract_model_coefficients <- function(models) {
  n_models <- length(models)
  coef_names <- names(coef(models[[1]]))
  
  # Initialize matrix
  all_coefs <- matrix(NA, nrow=n_models, ncol=length(coef_names))
  colnames(all_coefs) <- coef_names
  
  # Extract coefficients from each model
  for(i in 1:n_models) {
    all_coefs[i,] <- coef(models[[i]])
  }
  
  # Calculate mean coefficients
  mean_coefs <- colMeans(all_coefs)
  
  # Fungsi bantuan untuk mendapatkan knots
  get_knots <- function(var_name) {
    # Cari model dengan data lengkap
    for(model in models) {
      if(!is.null(model$x) && var_name %in% colnames(model$x)) {
        # Gunakan metode alternatif untuk mendapatkan knots
        x_var <- model$x[, var_name]
        knots <- quantile(x_var, probs = c(0.1, 0.5, 0.9))
        return(knots)
      }
    }
    return(NULL)
  }
  
  # Create coefficient object for JavaScript
  coeffs <- list(
    intercept = mean_coefs[1],  # Intercept
    rr_terms = mean_coefs[grep("^rr", names(mean_coefs), ignore.case=TRUE)],
    ureum_terms = mean_coefs[grep("^ureum", names(mean_coefs), ignore.case=TRUE)],
    ldl_terms = mean_coefs[grep("^ldl", names(mean_coefs), ignore.case=TRUE)],
    hdl_terms = mean_coefs[grep("^hdl", names(mean_coefs), ignore.case=TRUE)],
    sk_terms = mean_coefs[grep("^sk", names(mean_coefs), ignore.case=TRUE)],
    td_s_terms = mean_coefs[grep("^td_s", names(mean_coefs), ignore.case=TRUE)],
    riw_ht = mean_coefs[grep("^riw_ht", names(mean_coefs), ignore.case=TRUE)],
    usia = mean_coefs[grep("^usia", names(mean_coefs), ignore.case=TRUE)],
    bb_terms = mean_coefs[grep("^bb", names(mean_coefs), ignore.case=TRUE)],
    tg_terms = mean_coefs[grep("^tg", names(mean_coefs), ignore.case=TRUE)],
    gds_terms = mean_coefs[grep("^gds", names(mean_coefs), ignore.case=TRUE)],
    leukosit_terms = mean_coefs[grep("^leukosit", names(mean_coefs), ignore.case=TRUE)],
    hba1c = mean_coefs[grep("^hba1c", names(mean_coefs), ignore.case=TRUE)]
  )
  
  # Add knots for splines
  coeffs$knots <- list(
    rr = get_knots("RR"),
    ureum = get_knots("UREUM"),
    ldl = get_knots("LDL"),
    hdl = get_knots("HDL"),
    sk = get_knots("SK"),
    td_s = get_knots("TD_S"),
    bb = get_knots("BB"),
    tg = get_knots("TG"),
    gds = get_knots("GDS"),
    leukosit = get_knots("LEUKOSIT")
  )
  
  # Convert to JSON
  json_output <- jsonlite::toJSON(coeffs, pretty=TRUE, auto_unbox=TRUE)
  return(json_output)
}

In [303]:
coeffs_json <- extract_model_coefficients(ols_models)
write(coeffs_json, "./out/model_coefficients.json")

In [304]:
# Extract coefficients from OLS models
coeffs_json <- extract_model_coefficients(ols_models)

# Print preview
cat("Preview koefisien model:\n")
cat(substr(coeffs_json, 1, 1000), "...\n")

# Simpan ke file JSON
write(coeffs_json, "./out/model_coefficients.json")

# Print struktur untuk verifikasi
print("Struktur koefisien:")
print(str(jsonlite::fromJSON(coeffs_json)))

Preview koefisien model:
{
  "intercept": 4.2074,
  "rr_terms": [0.1351, 0.5245],
  "ureum_terms": [-0.408, 3.6079, -8.0549],
  "ldl_terms": [0.1116, -0.1111],
  "hdl_terms": [-1.1981, 0.9602],
  "sk_terms": [29.7954, 21.629, -99.1869],
  "td_s_terms": [-0.067, -0.2865],
  "riw_ht": -3.8358,
  "usia": 0.5517,
  "bb_terms": [-0.3226, -0.006],
  "tg_terms": [0.2441, -1.9115, 3.366],
  "gds_terms": [0.0355, -0.0198],
  "leukosit_terms": [4.051, -16.2034, 27.5119],
  "hba1c": -0.6342,
  "knots": {
    "rr": [18, 20, 28],
    "ureum": [20, 34, 61],
    "ldl": [66, 106, 186],
    "hdl": [28, 39, 53],
    "sk": [0.7, 1, 1.8],
    "td_s": [102.4, 131, 170.6],
    "bb": [50, 65, 75],
    "tg": [77, 136, 224.8],
    "gds": [89, 133, 248],
    "leukosit": [6.6488, 9.57, 15.668]
  }
} ...
[1] "Struktur koefisien:"
List of 15
 $ intercept     : num 4.21
 $ rr_terms      : num [1:2] 0.135 0.524
 $ ureum_terms   : num [1:3] -0.408 3.608 -8.055
 $ ldl_terms     : num [1:2] 0.112 -0.111
 $ hdl_terms   

### Ekstraksi Koefisien GAM

In [315]:
extract_gam_coefficients <- function(models) {
  n_models <- length(models)
  
  # Inisialisasi matrix untuk koefisien
  coef_names <- names(coef(models[[1]]))
  all_coefs <- matrix(NA, nrow=n_models, ncol=length(coef_names))
  colnames(all_coefs) <- coef_names
  
  # Extract koefisien dari setiap model
  for(i in 1:n_models) {
    all_coefs[i,] <- coef(models[[i]])
  }
  
  # Hitung rata-rata koefisien
  mean_coefs <- colMeans(all_coefs)
  
  # Struktur untuk menyimpan koefisien
  coeffs <- list(
    intercept = mean_coefs["(Intercept)"],
    rr_terms = mean_coefs[grep("^s\\(RR\\)", names(mean_coefs))],
    ureum_terms = mean_coefs[grep("^s\\(UREUM\\)", names(mean_coefs))],
    ldl_terms = mean_coefs[grep("^s\\(LDL\\)", names(mean_coefs))],
    hdl_terms = mean_coefs[grep("^s\\(HDL\\)", names(mean_coefs))],
    sk_terms = mean_coefs[grep("^s\\(SK\\)", names(mean_coefs))],
    td_s_terms = mean_coefs[grep("^s\\(TD_S\\)", names(mean_coefs))],
    bb_terms = mean_coefs[grep("^s\\(BB\\)", names(mean_coefs))],
    tg_terms = mean_coefs[grep("^s\\(TG\\)", names(mean_coefs))],
    gds_terms = mean_coefs[grep("^s\\(GDS\\)", names(mean_coefs))],
    leukosit_terms = mean_coefs[grep("^s\\(LEUKOSIT\\)", names(mean_coefs))],
    hba1c_terms = mean_coefs[grep("^s\\(HBA1C\\)", names(mean_coefs))],
    riw_ht = mean_coefs["RIW_HT"],
    usia = mean_coefs["USIA"]
  )
  
  # Dapatkan informasi basis dari model GAM
  model1 <- models[[1]]
  
  # Print untuk debugging
  print("Model smooth terms:")
  print(names(model1$smooth))
  
  # Extract basis information dengan penanganan error
  coeffs$basis_info <- list()
  for(s in model1$smooth) {
    var_name <- gsub("s\\((.+)\\)", "\\1", s$label)
    print(paste("Processing:", var_name))
    
    coeffs$basis_info[[var_name]] <- list(
      basis_type = s$bs.dim,
      df = s$df
    )
    
    # Coba dapatkan knots jika tersedia
    if(!is.null(s$knots)) {
      coeffs$basis_info[[var_name]]$knots <- s$knots
    }
  }
  
  # Konversi ke JSON dengan error handling
  tryCatch({
    json_output <- jsonlite::toJSON(coeffs, pretty=TRUE, auto_unbox=TRUE)
    return(json_output)
  }, error = function(e) {
    print("Error dalam konversi JSON:")
    print(e)
    return(NULL)
  })
}


In [317]:

# Jalankan fungsi dengan print debugging
print("Starting coefficient extraction...")
coeffs_json <- extract_gam_coefficients(gam_models)

if(!is.null(coeffs_json)) {
  # Simpan ke file
  write(coeffs_json, "./out/gam_coefficients.json")
  
  # Print preview
  cat("Preview koefisien model GAM:\n")
  cat(substr(coeffs_json, 1, 1000), "...\n")
} else {
  print("Failed to extract coefficients")
}

[1] "Starting coefficient extraction..."
[1] "Model smooth terms:"
NULL
[1] "Processing: RR"
[1] "Processing: UREUM"
[1] "Processing: LDL"
[1] "Processing: HDL"
[1] "Processing: SK"
[1] "Processing: TD_S"
[1] "Processing: BB"
[1] "Processing: TG"
[1] "Processing: GDS"
[1] "Processing: LEUKOSIT"
[1] "Processing: HBA1C"
Preview koefisien model GAM:
{
  "intercept": 19.5373,
  "rr_terms": [10.4185, 22.9244, 1.1568, 28.5055, 0.8165, 0.2375, -0.2986, 13.5266, -0.3231],
  "ureum_terms": [0.0012, 0.0065, 1.2808],
  "ldl_terms": [-1.3975, 10.9212, 1.0584, -5.3735, -0.05, 7.5951, -2.2597, 25.6734, 4.2388],
  "hdl_terms": [-1.6803, 0.0512, 0.2935, 1.1611, 0.2045, -1.0143, 0.4765, -12.0484, -2.7314],
  "sk_terms": [12.8099, 47.8652, -0.3477],
  "td_s_terms": [-2.824, 5.9761, -1.6805, -3.6287, 1.4193, 3.5005, 0.675, 18.0507, -3.6267],
  "bb_terms": [-0.0026, 0.0333, -0.001, -0.0517, 0.0007, -0.0445, -0.0078, -0.3589, -2.5616],
  "tg_terms": [0.6493, -0.8659, -2.0346],
  "gds_terms": [-2.3672, -8.3

In [309]:
summary(gam_models[[1]])


Family: gaussian 
Link function: identity 

Formula:
GENSINI ~ s(RR) + s(UREUM, k = 4) + s(LDL) + s(HDL) + s(SK, k = 4) + 
    s(TD_S) + RIW_HT + USIA + s(BB) + s(TG, k = 4) + s(GDS) + 
    s(LEUKOSIT, k = 4) + s(HBA1C)

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept)  15.7578    16.5095   0.954   0.3409  
RIW_HT       -5.4479     5.5044  -0.990   0.3234  
USIA          0.5769     0.2786   2.071   0.0395 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Approximate significance of smooth terms:
              edf Ref.df     F p-value  
s(RR)       1.928  2.148 1.716  0.1520  
s(UREUM)    1.000  1.000 0.053  0.8187  
s(LDL)      1.000  1.000 0.488  0.4854  
s(HDL)      2.463  3.159 3.271  0.0193 *
s(SK)       1.890  2.203 2.292  0.0993 .
s(TD_S)     1.000  1.000 1.359  0.2450  
s(BB)       1.000  1.000 1.478  0.2253  
s(TG)       1.000  1.000 2.559  0.1111  
s(GDS)      3.319  4.134 1.339  0.2662  
s(LEUKOSIT) 1.000  1.000 0.52

In [319]:
# Buat data frame baru dengan nilai default
newdata <- data.frame(
    RR = 28,
    UREUM = 58,
    LDL = 201,
    HDL = 68,
    SK = 0.6,
    TD_S = 96,
    RIW_HT = 0,  # 0 untuk "Tidak"
    USIA = 60,
    BB = 60,
    TG = 177,
    GDS = 245,
    LEUKOSIT = 8.813,
    HBA1C = 10.7
)

# Predict menggunakan model GAM
pred_gam <- predict(gam_models[[1]], newdata=newdata)
print(paste("GAM prediction:", pred_gam))

[1] "GAM prediction: 57.8648498744768"
